In [1]:
import pickle
import pandas as pd
import numpy as np
import nltk
import re
import gensim

from sklearn import model_selection 
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from nltk.util import ngrams
from gensim.utils import simple_preprocess
from gensim import corpora, models

pd.options.mode.chained_assignment = None 

/Users/bl4z3/miniconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
def rowIndex(sentence):
    tokens = word_tokenize(sentence.lower())
    return lda_model[dictionary.doc2bow(tokens)] 

def generateNgrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [3]:
allSessions = pickle.load( open( "../Data/DataSets/SQS/SQS.p", "rb" ) )
# allSessionsQ = allSessions.loc[allSessions['type']=='Q']
allSessionsA = pickle.load( open( "../Data/DataSets/SQS/SQSA.p", "rb" ) )
allSessionsNA = pickle.load( open( "../Data/DataSets/SQS/SQSNA.p", "rb" ) )
ovaResults = []


In [4]:
list(allSessionsA)

['How did paul die from fast in the fouris',
 'Fire belly toad',
 'Now im stressed out',
 'fast in the fouris',
 'How many phones sell in a day',
 'How long do toads live',
 'amozan',
 'Fast and the ferious',
 "Dominic's early life in fast an ferious",
 'what is a cheeta',
 'The song at the end of the fast and ferious 7 movie',
 'The Fast and ferious 7 movie',
 'When is the Fast and ferious 9 coming in theaters',
 'hew was the 2nd president',
 'movies for to day at the edwads movie theater',
 'Who is the voice of rupnnzel',
 'tortoises',
 'Hey I just met you',
 'Jurassic world',
 'How many langenges was Jurassic world made in',
 'TTM',
 'Google classroom',
 '23rd president',
 'cute girls hairstyles',
 'pintrest',
 'Fantastic hair doos',
 'Character from monster high',
 'example of a leamer',
 'taler swift',
 'monster high move boo york boo york',
 'why does the poler bear have white fur',
 'Whoh played Ioin hide from transformers',
 'how the chetta has spots',
 'florider',
 'tranksform

In [5]:
newList = {}
newList['entry']=[]
newList['class']=[]

for session in allSessionsA:
    newList['entry'].append(session)
    #newList['tag'].append('kids')
    newList['class'].append(1)


for session in allSessionsNA:
    newList['entry'].append(session)
    newList['class'].append(0)

data = pd.DataFrame(newList)

In [6]:
data

,entry,class
0,How did paul die from fast in the fouris,1
1,Fire belly toad,1
2,Now im stressed out,1
3,fast in the fouris,1
4,How many phones sell in a day,1
...,...,...
1500,apply for a State University of New York college,0
1501,supreme court on banning guns,0
1502,philadelphia nyc distance,0
1503,themed road trips,0


# Majority

In [7]:
nSplits = 5
kfold = KFold(n_splits=nSplits, random_state=20210530, shuffle=True)

data['prediction'] = 0

tnOva = 0
fpOva = 0
fnOva = 0
tpOva = 0
testAccOva = 0
outputMaj = []
trI = []
tI = []
for test_index,train_index in kfold.split(data):
    trI.append(train_index)
    tI.append(test_index)
    test = data.iloc[test_index]
    outputMaj.append(test['prediction'].tolist())
    testAccOva +=  accuracy_score(test['class'], test['prediction'])
    tn, fp, fn, tp = confusion_matrix( test['class'], test['prediction']).ravel()
    tnOva += tn
    fpOva += fp
    fnOva += fn
    tpOva += tp
pickle.dump( outputMaj, open( "Pickles/OutputMajoritySQS.p", "wb" ) )
majorityResults = ['Majority', round(testAccOva/nSplits,3), tnOva/nSplits, fpOva/nSplits, round(((tnOva/nSplits)/((tnOva/nSplits)+ (fpOva/nSplits))),3), fnOva/nSplits, tpOva/nSplits,round(((tpOva/nSplits)/((tpOva/nSplits)+ (fnOva/nSplits))),3),]

In [8]:
majorityResults

['Majority', 0.8, 963.2, 0.0, 1.0, 240.8, 0.0, 0.0]

In [9]:
ovaResults.append(majorityResults)

# Duarte Torres

In [1]:
arch = pickle.load( open( "../Data/DataSets/SQS/SQSA.p", "rb" ) )
notArch = pickle.load( open( "../Data/DataSets/SQS/SQSNA.p", "rb" ) )

from datetime import datetime
documents = []
sessionDuration = []
seshClass = []
for session in arch:
    #print(session)
    sites = []
    doc = ""
    documents.append(sites)
    length = 0
    sessionDuration.append(length)
    seshClass.append(1)
for session in notArch:
    #print(session)
    sites = []
    doc = ""
    documents.append(sites)
    length = 0
    sessionDuration.append(length)
    seshClass.append(0)

NameError: name 'pickle' is not defined

In [11]:
data = pd.DataFrame(data=documents)
data = data[data.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
data = pd.DataFrame(data, columns=['sites'])

data['duration'] = sessionDuration
data['class'] = seshClass

In [12]:
data

,sites,duration,class
0,,0,1
1,,0,1
2,,0,1
3,,0,1
4,,0,1
...,...,...,...
1500,,0,0
1501,,0,0
1502,,0,0
1503,,0,0


In [13]:
Sites = data['sites'].tolist()
Duration = data['duration'].tolist()

durationClassification = []

for sD in Duration:
    try:
        if float(sD) <= (60*30):
            durationClassification.append(1)
        else:
            durationClassification.append(0)    
    except:
        pass

In [14]:
import csv

kids_sites = []
with open('../Data/DataSources/DMOZ/URL Classification.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    line_count = -1
    for row in csv_reader:
        
        ## consume first line
        if( 'Kids' in row[2]):
            #print(row)
            kids_sites.append(row[1])

In [15]:
from tqdm import tqdm

siteClassification = []
with tqdm(total=len(Sites)) as pbar:
    for visitedSites in Sites:
        found = 0
        for kidsSite in kids_sites:
            if kidsSite in visitedSites:
                siteClassification.append(1)
                found = 1
                break
        if found == 0:
            siteClassification.append(0)
        pbar.update()
        

100%|██████████| 1505/1505 [00:39<00:00, 38.08it/s]


In [16]:
data['durationClassification'] = durationClassification
data['siteClassification'] = siteClassification
data['prediction'] = data['durationClassification'] & data['siteClassification']

In [17]:
tnOva = 0
fpOva = 0
fnOva = 0
tpOva = 0
testAccOva = 0
outputDT = []
for test_index, train_index in kfold.split(data):
    test = data.iloc[test_index]
    outputDT.append( test['prediction'].tolist())
    testAccOva +=  accuracy_score(test['class'], test['prediction'])
    tn, fp, fn, tp = confusion_matrix(test['class'], test['prediction']).ravel()
    tnOva += tn
    fpOva += fp
    fnOva += fn
    tpOva += tp

In [18]:
pickle.dump( outputDT, open( "Pickles/OutputRuleSQS.p", "wb" ) )

In [19]:
ruleResults = ['Rule', round(testAccOva/5,3), tnOva/nSplits, fpOva/nSplits, round(((tnOva/nSplits)/((tnOva/nSplits)+ (fpOva/nSplits))),3), fnOva/nSplits, tpOva/nSplits,round(((tpOva/nSplits)/((tpOva/nSplits)+ (fnOva/nSplits))),3),]

In [20]:
ruleResults

['Rule', 0.8, 963.2, 0.0, 1.0, 240.8, 0.0, 0.0]

In [21]:
ovaResults.append(ruleResults)

# Text Based

In [22]:
allSessions = pickle.load( open( "Pickles/SWCNoTune.p", "rb" ) )
allSessionsQ = allSessions.loc[allSessions['type']=='Q']
allSessionsA = allSessionsQ.loc[allSessionsQ['class']==1].groupby('sID')['query'].apply(pd.Series.tolist).tolist()
allSessionsNA = allSessionsQ.loc[allSessionsQ['class']==0].groupby('sID')['query'].apply(pd.Series.tolist).tolist()


SQSA = np.asarray(pickle.load( open( "../Data/DataSets/SQS/SQSA.p", "rb" ) ))
SQSNA = np.asarray(pickle.load( open( "../Data/DataSets/SQS/SQSNA.p", "rb" ) ))

In [23]:
newList = {}
newList['entry']=[]
newList['tag']=[]


for session in allSessionsA:
    string = ""
    for query in session:
        string += query + " "
    newList['entry'].append(string)
    #newList['tag'].append('kids')
    newList['tag'].append(1)


for session in allSessionsNA:
    string = ""
    for query in session:
        string += query + " "
    newList['entry'].append(string)
    newList['tag'].append(0)

newList = pd.DataFrame(data = newList)

In [24]:
rNum = 20210530    
rng = np.random.RandomState(rNum)

newList2 = {}
newList2['entry']=[]
newList2['tag']=[]

for x in range((len(SQSA))+(len(SQSNA))):
    if( x < (len(SQSA))):
        newList2['tag'].append(1)
    else:
        newList2['tag'].append(0)


for x in range((len(SQSA))+(len(SQSNA))):
    if( x < (len(SQSA))):
        newList2['entry'].append(SQSA[x])

    else:
        newList2['entry'].append(SQSNA[x-(len(SQSA))])

data2 = pd.DataFrame(data = newList2)

In [25]:
tnOva = 0
fpOva = 0
fnOva = 0
tpOva = 0
testAccOva = 0

outputTamSing = []
for train_index, test_index in kfold.split(data2):

    Test_X, Train_X = data2.iloc[train_index]['entry'], data2.iloc[test_index]['entry']
    Test_Y, Train_Y = data2.iloc[train_index]['tag'], data2.iloc[test_index]['tag']
    Encoder = LabelEncoder()


    rng = np.random.RandomState(rNum)
    test = newList.sample(frac=0.20, random_state=rng)
    train_mask = pd.Series(True, index=newList.index)
    train_mask[test.index] = False
    train = newList[train_mask].copy()
    train.head()
    Train_X = pd.concat([Train_X, train['entry']])
    Train_Y = pd.concat([Train_Y, train['tag']])

    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(Train_Y)
    Test_Y = Encoder.fit_transform(Test_Y)
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(3, 3))
    vectorizer.fit_transform(newList['entry'])
    Train_X_Tfidf = vectorizer.transform(Train_X)
    Test_X_Tfidf = vectorizer.transform(Test_X)
    SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    outputTamSing.append(predictions_SVM)
    testAccOva += accuracy_score(predictions_SVM, Test_Y)
    tn, fp, fn, tp = confusion_matrix(Test_Y, predictions_SVM).ravel()
    tnOva += tn
    fpOva += fp
    fnOva += fn
    tpOva += tp
    rNum+=1

In [26]:
pickle.dump( outputTamSing, open( "Pickles/OutputTextSQS.p", "wb" ) )

In [27]:
textResults = ['Text', round(testAccOva/nSplits,3), tnOva/nSplits, fpOva/nSplits, round(((tnOva/nSplits)/((tnOva/nSplits)+ (fpOva/nSplits))),3), fnOva/nSplits, tpOva/nSplits,round(((tpOva/nSplits)/((tpOva/nSplits)+ (fnOva/nSplits))),3),]

In [28]:
textResults

['Text', 0.808, 962.8, 0.4, 1.0, 230.2, 10.6, 0.044]

In [29]:
ovaResults.append(textResults)

# Santosh

In [30]:
allSessions = pickle.load( open( "Pickles/SWCNoTune.p", "rb" ) )
allSessionsQ = allSessions.loc[allSessions['type']=='Q']
allSessionsA = allSessionsQ.loc[allSessionsQ['class']==1].groupby('sID')['query'].apply(pd.Series.tolist).tolist()
allSessionsNA = allSessionsQ.loc[allSessionsQ['class']==0].groupby('sID')['query'].apply(pd.Series.tolist).tolist()


SQSA = np.asarray(pickle.load( open( "../Data/DataSets/SQS/SQSA.p", "rb" ) ))
SQSNA = np.asarray(pickle.load( open( "../Data/DataSets/SQS/SQSNA.p", "rb" ) ))

documents = []
classList = []
for session in allSessionsA:
    
    doc = ""
    for query in session:
        doc += " " + query
    documents.append(doc)
    classList.append(1)
    
for session in allSessionsNA:
    
    doc = ""
    for query in session:
        doc += " " + query
    documents.append(doc)
    classList.append(0)

In [31]:
newList = pd.DataFrame(data = newList)

In [32]:
documents = []
classList = []
for session in allSessionsA:
    
    doc = ""
    for query in session:
        doc += " " + query
    documents.append(doc)
    classList.append(1)
    
for session in allSessionsNA:
    
    doc = ""
    for query in session:
        doc += " " + query
    documents.append(doc)
    classList.append(0)

In [33]:
data = pd.DataFrame(documents, columns = ['text'])
posData = []
for document in documents:
    text = nltk.word_tokenize(document)
    tags = np.array(nltk.pos_tag(text)).flatten()
    posData.append(tags[1::2])

In [34]:
data['pos'] = posData

In [35]:
docUni = []
docBi = []
docTri = []

for document in documents:
    doc = generateNgrams(document,1)
    docUni.append(doc)
    
for document in documents:
    doc = generateNgrams(document,2)
    docBi.append(doc)
    
for document in documents:
    doc = generateNgrams(document,3)
    docTri.append(doc)
    
data['uniWord'] = docUni
data['biWord'] = docBi
data['triWord']= docTri

In [36]:
posMod = []

for pos in posData: 
    string = []
    for entry in pos:
        string += str(entry) + " "
    posMod.append("".join(string))

    
posUni = []
posBi = []
posTri = []

for document in posMod:
    doc = generateNgrams(document,1)
    posUni.append(doc)

for document in posMod:
    doc = generateNgrams(document,2)
    posBi.append(doc)

for document in posMod:
    doc = generateNgrams(document,3)
    posTri.append(doc)  
    
data['uniPos'] = posUni
data['biPos'] = posBi
data['triPos']= posTri
data['posMod'] = posMod
data['class'] = classList

In [37]:
import collections

text = "".join(data['text'].tolist())
token = word_tokenize(text)

unigrams = list(ngrams(token, 1))
uniwords = collections.Counter(unigrams)
n = len(uniwords)-40000
stopWordsUni = uniwords.most_common()[:-n-1:-1]

bigrams = list(ngrams(token, 2))
biwords = collections.Counter(bigrams)
n = len(biwords)-40000
stopWordsBi =  biwords.most_common()[:-n-1:-1]

trigrams = list(ngrams(token, 3))
triwords = collections.Counter(trigrams)
n = len(triwords)-40000
stopWordsTri = triwords.most_common()[:-n-1:-1]

In [38]:
documents = []
classList = []

for query in SQSA:
    doc = ""
    doc += " " + query
    documents.append(doc)
    classList.append(1)
 
for query in SQSNA:
    doc = ""   
    doc += " " + query
    documents.append(doc)
    classList.append(0)

data2 = pd.DataFrame(documents, columns = ['text'])

posData = []
for document in documents:
    text = nltk.word_tokenize(document)
    tags = np.array(nltk.pos_tag(text)).flatten()
    posData.append(tags[1::2])
    
data2['pos'] = posData

docUni = []
docBi = []
docTri = []

for document in documents:
    doc = generateNgrams(document,1)
    docUni.append(doc)
    
for document in documents:
    doc = generateNgrams(document,2)
    docBi.append(doc)
    
for document in documents:
    doc = generateNgrams(document,3)
    docTri.append(doc)
    
data2['uniWord'] = docUni
data2['biWord'] = docBi
data2['triWord']= docTri

posMod = []

for pos in posData: 
    string = []
    for entry in pos:
        string += str(entry) + " "
    posMod.append("".join(string))

    
posUni = []
posBi = []
posTri = []

for document in posMod:
    doc = generateNgrams(document,1)
    posUni.append(doc)

for document in posMod:
    doc = generateNgrams(document,2)
    posBi.append(doc)

for document in posMod:
    doc = generateNgrams(document,3)
    posTri.append(doc)  
    
data2['uniPos'] = posUni
data2['biPos'] = posBi
data2['triPos']= posTri
data2['posMod'] = posMod
data2['class'] = classList
data2['numQ'] = 1

In [39]:
data2

,text,pos,uniWord,biWord,triWord,uniPos,biPos,triPos,posMod,class,numQ
0,How did paul die from fast in the fouris,"[WRB, VBD, VB, NN, IN, NN, IN, DT, NN]","[how, did, paul, die, from, fast, in, the, fou...","[how did, did paul, paul die, die from, from f...","[how did paul, did paul die, paul die from, di...","[wrb, vbd, vb, nn, in, nn, in, dt, nn]","[wrb vbd, vbd vb, vb nn, nn in, in nn, nn in, ...","[wrb vbd vb, vbd vb nn, vb nn in, nn in nn, in...",WRB VBD VB NN IN NN IN DT NN,1,1
1,Fire belly toad,"[NNP, RB, VBD]","[fire, belly, toad]","[fire belly, belly toad]",[fire belly toad],"[nnp, rb, vbd]","[nnp rb, rb vbd]",[nnp rb vbd],NNP RB VBD,1,1
2,Now im stressed out,"[RB, RB, VBN, RP]","[now, im, stressed, out]","[now im, im stressed, stressed out]","[now im stressed, im stressed out]","[rb, rb, vbn, rp]","[rb rb, rb vbn, vbn rp]","[rb rb vbn, rb vbn rp]",RB RB VBN RP,1,1
3,fast in the fouris,"[RB, IN, DT, NN]","[fast, in, the, fouris]","[fast in, in the, the fouris]","[fast in the, in the fouris]","[rb, in, dt, nn]","[rb in, in dt, dt nn]","[rb in dt, in dt nn]",RB IN DT NN,1,1
4,How many phones sell in a day,"[WRB, JJ, NNS, VBP, IN, DT, NN]","[how, many, phones, sell, in, a, day]","[how many, many phones, phones sell, sell in, ...","[how many phones, many phones sell, phones sel...","[wrb, jj, nns, vbp, in, dt, nn]","[wrb jj, jj nns, nns vbp, vbp in, in dt, dt nn]","[wrb jj nns, jj nns vbp, nns vbp in, vbp in dt...",WRB JJ NNS VBP IN DT NN,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1500,apply for a State University of New York college,"[NN, IN, DT, NNP, NNP, IN, NNP, NNP, NN]","[apply, for, a, state, university, of, new, yo...","[apply for, for a, a state, state university, ...","[apply for a, for a state, a state university,...","[nn, in, dt, nnp, nnp, in, nnp, nnp, nn]","[nn in, in dt, dt nnp, nnp nnp, nnp in, in nnp...","[nn in dt, in dt nnp, dt nnp nnp, nnp nnp in, ...",NN IN DT NNP NNP IN NNP NNP NN,0,1
1501,supreme court on banning guns,"[JJ, NN, IN, VBG, NNS]","[supreme, court, on, banning, guns]","[supreme court, court on, on banning, banning ...","[supreme court on, court on banning, on bannin...","[jj, nn, in, vbg, nns]","[jj nn, nn in, in vbg, vbg nns]","[jj nn in, nn in vbg, in vbg nns]",JJ NN IN VBG NNS,0,1
1502,philadelphia nyc distance,"[NN, NN, NN]","[philadelphia, nyc, distance]","[philadelphia nyc, nyc distance]",[philadelphia nyc distance],"[nn, nn, nn]","[nn nn, nn nn]",[nn nn nn],NN NN NN,0,1
1503,themed road trips,"[VBN, NN, NNS]","[themed, road, trips]","[themed road, road trips]",[themed road trips],"[vbn, nn, nns]","[vbn nn, nn nns]",[vbn nn nns],VBN NN NNS,0,1


In [40]:

FlatSingle = pd.DataFrame(data= data2)
rn = 20210530

nSplits = 5
tnOva = 0
fpOva = 0
fnOva = 0
tpOva = 0
testAccOva = 0
x = 0
accByNum = []
kfold = KFold(n_splits=nSplits, random_state=20210530, shuffle=True)
outputSanSing = []
for train_index, test_index in kfold.split(data2):
    

    train = data2.iloc[test_index]
    test = data2.iloc[train_index]
    
    rng = np.random.RandomState(rn)
    testSing = data.sample(frac=0.20, random_state=rng)
    train_mask = pd.Series(True, index=data.index)
    train_mask[testSing.index] = False
    trainSing = data[train_mask].copy()


#     print(len(trainSing))
    train.head()
    train = pd.concat([train,trainSing])
#     print(len(train))
    #print(len(test))
    subtest = train.sample(frac=0.20, random_state=rng)
    subtrain_mask = pd.Series(True, index=train.index)
    subtrain_mask[subtest.index] = False
    subtrain = train[subtrain_mask].copy()
    subtrain.head()

    print('start')

    subtest = train.sample(frac=0.20, random_state=rng)
    subtrain_mask = pd.Series(True, index=train.index)
    subtrain_mask[subtest.index] = False
    subtrain = train[subtrain_mask].copy()
    subtrain.head()
#     print(subtest)
    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(subtrain['class'])
    Test_Y = Encoder.fit_transform(subtest['class'])

    vectorizer = CountVectorizer(stop_words=stopWordsUni)
    vectorizer.fit_transform(train['text'])
    Train_X_Tfidf = vectorizer.transform(subtrain['text'])
    Test_X_Tfidf = vectorizer.transform(subtest['text'])

    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM_Uni = SVM.predict(Test_X_Tfidf)

    Test_X_Tfidf = vectorizer.transform(test['text'])
    test_predictions_SVM_uni = SVM.predict(Test_X_Tfidf)

    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(subtrain['class'])
    Test_Y = Encoder.fit_transform(subtest['class'])

    vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2),stop_words=stopWordsBi)
    vectorizer.fit_transform(train['text'])
    Train_X_Tfidf = vectorizer.transform(subtrain['text'])
    Test_X_Tfidf = vectorizer.transform(subtest['text'])

    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', )
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM_bi = SVM.predict(Test_X_Tfidf)

    Test_X_Tfidf = vectorizer.transform(test['text'])
    test_predictions_SVM_bi = SVM.predict(Test_X_Tfidf)

    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(subtrain['class'])
    Test_Y = Encoder.fit_transform(subtest['class'])

    vectorizer = CountVectorizer(analyzer='word', ngram_range=(3, 3), stop_words=stopWordsTri)
    vectorizer.fit_transform(train['text'])
    Train_X_Tfidf = vectorizer.transform(subtrain['text'])
    Test_X_Tfidf = vectorizer.transform(subtest['text'])

    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM_tri = SVM.predict(Test_X_Tfidf)

    Test_X_Tfidf = vectorizer.transform(test['text'])
    test_predictions_SVM_tri = SVM.predict(Test_X_Tfidf)

    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(subtrain['class'])
    Test_Y = Encoder.fit_transform(subtest['class'])

    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train['posMod'])
    Train_X_Tfidf = vectorizer.transform(subtrain['posMod'])
    Test_X_Tfidf = vectorizer.transform(subtest['posMod'])

    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM_pos_uni = SVM.predict(Test_X_Tfidf)


    Test_X_Tfidf = vectorizer.transform(test['posMod'])
    test_predictions_SVM_pos_uni = SVM.predict(Test_X_Tfidf)

    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(subtrain['class'])
    Test_Y = Encoder.fit_transform(subtest['class'])

    vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
    vectorizer.fit_transform(train['posMod'])
    Train_X_Tfidf = vectorizer.transform(subtrain['posMod'])
    Test_X_Tfidf = vectorizer.transform(subtest['posMod'])

    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM_pos_bi = SVM.predict(Test_X_Tfidf)


    Test_X_Tfidf = vectorizer.transform(test['posMod'])
    test_predictions_SVM_pos_bi = SVM.predict(Test_X_Tfidf)

    Encoder = LabelEncoder()
    Train_Y = Encoder.fit_transform(subtrain['class'])
    Test_Y = Encoder.fit_transform(subtest['class'])

    vectorizer = CountVectorizer(analyzer='word', ngram_range=(3, 3))
    vectorizer.fit_transform(train['posMod'])
    Train_X_Tfidf = vectorizer.transform(subtrain['posMod'])
    Test_X_Tfidf = vectorizer.transform(subtest['posMod'])

    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
    predictions_SVM_pos_tri = SVM.predict(Test_X_Tfidf)



    Test_X_Tfidf = vectorizer.transform(test['posMod'])
    test_predictions_SVM_pos_tri = SVM.predict(Test_X_Tfidf)

    LDAtest = train.sample(frac=0.4, random_state=rng)
    LDAtrain_mask = pd.Series(True, index=train.index)
    LDAtrain_mask[LDAtest.index] = False
    LDAtrain = train[LDAtrain_mask].copy()


    processed_docs = LDAtrain['uniWord']
    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = models.LdaModel(corpus, num_topics=200, \
                                      id2word=dictionary, \
                                      passes=4, alpha=[0.01]*200, \
                                      eta=[0.01]*len(dictionary.keys()))

    subtest['ldaOneOverall'] = subtest['text'].apply(lambda x: rowIndex(x))
    test['ldaOneOverall'] = test['text'].apply(lambda x: rowIndex(x))
    LDAtest['ldaOneOverall'] = LDAtest['text'].apply(lambda x: rowIndex(x))

    ldaOOList = LDAtest['ldaOneOverall'].tolist()

    emptyList200 = []

    for x in range(200):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)


    LDAtest['ldaOneOverallPad'] =  ldaAllPad

    ldaOOList = test['ldaOneOverall'].tolist()

    emptyList200 = []

    for x in range(200):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    test['ldaOneOverallPad'] =  ldaAllPad

    ldaOOList = subtest['ldaOneOverall'].tolist()

    emptyList200 = []

    for x in range(200):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    subtest['ldaOneOverallPad'] =  ldaAllPad

    processed_docs = LDAtrain.loc[LDAtrain['class'] ==1]['uniWord']
    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = models.LdaModel(corpus, num_topics=100, \
                                      id2word=dictionary, \
                                      passes=4, alpha=[0.01]*100, \
                                      eta=[0.01]*len(dictionary.keys()))

    subtest['ldaOneTrue'] = subtest['text'].apply(lambda x: rowIndex(x))
    test['ldaOneTrue'] = test['text'].apply(lambda x: rowIndex(x))
    LDAtest['ldaOneTrue'] = LDAtest['text'].apply(lambda x: rowIndex(x))

    ldaOOList = LDAtest['ldaOneTrue'].tolist()

    emptyList200 = []

    for x in range(100):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    LDAtest['ldaOneTrue'] =  ldaAllPad

    ldaOOList = test['ldaOneTrue'].tolist()

    emptyList200 = []

    for x in range(100):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    test['ldaOneTrue'] =  ldaAllPad

    ldaOOList = subtest['ldaOneTrue'].tolist()

    emptyList200 = []

    for x in range(100):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    subtest['ldaOneTrue'] =  ldaAllPad

    processed_docs = LDAtrain.loc[LDAtrain['class'] ==0]['uniWord']
    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = models.LdaModel(corpus, num_topics=100, \
                                      id2word=dictionary, \
                                      passes=4, alpha=[0.01]*100, \
                                      eta=[0.01]*len(dictionary.keys()))

    subtest['ldaOneFalse'] = subtest['text'].apply(lambda x: rowIndex(x))
    test['ldaOneFalse'] = test['text'].apply(lambda x: rowIndex(x))
    LDAtest['ldaOneFalse'] = LDAtest['text'].apply(lambda x: rowIndex(x))

    ldaOOList = LDAtest['ldaOneFalse'].tolist()

    emptyList200 = []

    for x in range(100):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    LDAtest['ldaOneFalse'] =  ldaAllPad

    ldaOOList = test['ldaOneFalse'].tolist()

    emptyList200 = []

    for x in range(100):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    test['ldaOneFalse'] =  ldaAllPad

    ldaOOList = subtest['ldaOneFalse'].tolist()

    emptyList200 = []

    for x in range(100):
        emptyList200.append((0))

    ldaAllPad = []
    for entry in ldaOOList:
        if entry:
            newList = emptyList200.copy()
            for tupleItem in entry:
                newList[tupleItem[0]] = tupleItem[1]
            ldaAllPad.append(newList)
        else:
            ldaAllPad.append(emptyList200)

    subtest['ldaOneFalse'] =  ldaAllPad

    LDAtest['LDAClassOne'] = LDAtest['ldaOneTrue'] + LDAtest['ldaOneFalse']
    test['LDAClassOne'] = test['ldaOneTrue'] + test['ldaOneFalse'] 
    subtest['LDAClassOne'] = subtest['ldaOneTrue'] + subtest['ldaOneFalse'] 

    LDAtest['LDAallOne'] = LDAtest['ldaOneOverallPad'] + LDAtest['LDAClassOne']
    test['LDAallOne'] = test['ldaOneOverallPad'] + test['LDAClassOne'] 
    subtest['LDAallOne'] = subtest['ldaOneOverallPad'] + subtest['LDAClassOne'] 

    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(random_state=0, solver = 'liblinear').fit(list(LDAtest['ldaOneOverallPad']),LDAtest['class'])

    resultsOne = clf.predict(list(subtest['ldaOneOverallPad']))
    testresultsOne = clf.predict(list(test['ldaOneOverallPad']))

    clf = LogisticRegression(random_state=0, solver = 'liblinear').fit(list(LDAtest['LDAClassOne']),LDAtest['class'])

    resultsTwo = clf.predict(list(subtest['LDAClassOne']))
    testresultsTwo = clf.predict(list(test['LDAClassOne']))

    clf = LogisticRegression(random_state=0, solver = 'liblinear').fit(list(LDAtest['LDAallOne']),LDAtest['class'])

    resultsThree = clf.predict(list(subtest['LDAallOne']))
    testresultsThree = clf.predict(list(test['LDAallOne']))

    subtest['SVMUni'] = predictions_SVM_Uni
    subtest['SVMBi'] = predictions_SVM_bi
    subtest['SVMTri'] = predictions_SVM_tri
    subtest['SVMUniPOS'] = predictions_SVM_pos_uni
    subtest['SVMBiPOS'] = predictions_SVM_pos_bi
    subtest['SVMTriPOS'] = predictions_SVM_pos_tri
    subtest['SVMUniLDA'] = resultsOne
    subtest['SVMBiLDA'] = resultsTwo
    subtest['SVMTriLDA'] = resultsThree

    test['SVMUni'] = test_predictions_SVM_uni
    test['SVMBi'] = test_predictions_SVM_bi
    test['SVMTri'] = test_predictions_SVM_tri
    test['SVMUniPOS'] = test_predictions_SVM_pos_uni
    test['SVMBiPOS'] = test_predictions_SVM_pos_bi
    test['SVMTriPOS'] = test_predictions_SVM_pos_tri
    test['SVMUniLDA'] = testresultsOne
    test['SVMBiLDA'] = testresultsTwo
    test['SVMTriLDA'] = testresultsThree


    feat_cols = [
        'SVMUni',
        'SVMBi',
        'SVMTri',
        'SVMUniPOS',
        'SVMBiPOS',
        'SVMTriPOS',
        'SVMUniLDA',
        'SVMBiLDA',
        'SVMTriLDA',


    ]

    out_col = 'class'

    train_x = subtest[feat_cols]
    train_y = subtest[out_col]
    test_x = test[feat_cols]
    test_y = test[out_col]

    pure_pipeL2 = Pipeline([
        ('standardize', StandardScaler()),
        ('classify', DecisionTreeClassifier(criterion='entropy',random_state= 20210518, class_weight = 'balanced'))
        ])

    pure_pipeL2.fit(train_x, train_y)


    from sklearn.metrics import confusion_matrix
    prediction = pure_pipeL2.predict(test_x)
    testAccOva += accuracy_score(test_y, prediction)
    outputSanSing.append(prediction)
    SanAQtn, SanAQfp, SanAQfn, SanAQtp = confusion_matrix(test_y, prediction).ravel()
    tnOva += SanAQtn
    fpOva += SanAQfp
    fnOva += SanAQfn
    tpOva += SanAQtp
    rn+=1
    x+=1


1204
start
1204
start
1204
start
1204
start
1204
start


In [41]:
pickle.dump( outputSanSing, open( "Pickles/OutputMultiFeatureSQS.p", "wb" ) )

In [42]:
outputSanSing

[array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 1, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 1, ..., 0, 0, 0])]

In [43]:
multiFeatureResults = ['MultiFeature', round(testAccOva/nSplits,3), tnOva/nSplits, fpOva/nSplits, round(((tnOva/nSplits)/((tnOva/nSplits)+ (fpOva/nSplits))),3), fnOva/nSplits, tpOva/nSplits,round(((tpOva/nSplits)/((tpOva/nSplits)+ (fnOva/nSplits))),3),]

In [44]:
print(multiFeatureResults)

['MultiFeature', 0.832, 955.2, 8.0, 0.992, 194.8, 46.0, 0.191]


In [45]:
ovaResults.append(multiFeatureResults )

# Nemati

In [46]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
nSplits = 5
kfold = KFold(n_splits=nSplits, random_state=20210530, shuffle=True)

allSessions = pickle.load( open( "Pickles/SWCNoTune.p", "rb" ) )
allSessionsQ = allSessions.loc[allSessions['type']=='Q']
allSessionsK = allSessionsQ.loc[allSessionsQ['class']==1]
allSessionsA = allSessionsQ.loc[allSessionsQ['class']==0]

kidsAll = allSessionsK.groupby('sID')['query'].apply(pd.Series.tolist).tolist()
TRECS =  allSessionsA.groupby('sID')['query'].apply(pd.Series.tolist).tolist()
kidsSolo = np.asarray(pickle.load( open( "../Data/DataSets/SQS/SQSA.p", "rb" ) ))
millQ = np.asarray(pickle.load( open( "../Data/DataSets/SQS/SQSNA.p", "rb" ) ))

newList = {}
newList['entry']=[]
newList['class']=[]
newList['numQ']=[]

for session in kidsAll:
    string = ""
    numQ = 0
    for query in session:
        string += query + " "
        numQ += 1
    newList['entry'].append(string)
    #newList['tag'].append('kids')
    newList['class'].append(1)
    newList['numQ'].append(numQ)


for session in TRECS:
    string = ""
    numQ = 0
    for query in session:
        string += query + " "
        numQ += 1
    newList['entry'].append(string)
    #newList['tag'].append('kids')
    newList['class'].append(0)
    newList['numQ'].append(numQ)
data = pd.DataFrame(newList)

In [47]:
Test_Y1 = []
for x in range((301 + 1204)):
    if( x < (301)):
        Test_Y1.append(1)
    else:
        Test_Y1.append(0)
Test_X2 = []
for x in range((301 + 1204)):
    if( x < (301)):
        Test_X2.append(kidsSolo[x])
    else:
        Test_X2.append(millQ[x-301])
        
single = pd.DataFrame(data =[Test_Y1,Test_X2]).T
single.rename(columns ={0:'class', 1:'entry'}, inplace = True)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [49]:
estimators = [
    ('lr', make_pipeline(CountVectorizer(),
                           LogisticRegressionCV(solver = 'liblinear', max_iter=1000))),
    ('nb', make_pipeline(TfidfVectorizer(),
                          MultinomialNB(alpha = .13))),
    ('gb', make_pipeline(CountVectorizer(),
                            GradientBoostingClassifier(learning_rate = .2, max_depth = 2))),
    ('mlp', make_pipeline(CountVectorizer(),
                           MLPClassifier(hidden_layer_sizes= 21, max_iter=1500, shuffle=False, tol=0.012 ))),   
 ]

In [50]:
eclf1 = VotingClassifier(estimators=estimators, voting='hard')

tnOva = 0
fpOva = 0
fnOva = 0
tpOva = 0
testAccOva = 0
accByNum = []
outputNem = []
outputQ = []
rn = 20210530
rng = np.random.RandomState(rn)
tnOva = 0
fpOva = 0
fnOva = 0
tpOva = 0
testAccOva = 0
outputNemSing = []
x = 0
for train_index, test_index in kfold.split(single):

    rng = np.random.RandomState(rn)
    test = data.sample(frac=0.20, random_state=rng)
    train_mask = pd.Series(True, index=data.index)
    train_mask[test.index] = False
    train = data[train_mask].copy()
    train.head()

    X_train, X_test = single.iloc[train_index]['entry'], single.iloc[test_index]['entry']
    y_train, y_test = single.iloc[train_index]['class'], single.iloc[test_index]['class']
    #print(X_test)
    X_test = pd.concat([X_test, test['entry']])
    y_test = pd.concat([y_test, test['class']])
    #print(X_test.tolist())
    eclf1 = eclf1.fit(X_test.tolist(), y_test.tolist())
    prediction = eclf1.predict(X_train)
    outputNemSing.append(prediction)
    testAccOva +=  accuracy_score(prediction, y_train.tolist())
    tn, fp, fn, tp = confusion_matrix( y_train.tolist(), prediction).ravel()
    tnOva += tn
    fpOva += fp
    fnOva += fn
    tpOva += tp
    rn+=1
    x+=1

In [51]:
pickle.dump( outputNemSing, open( "Pickles/OutputMultiModelSQS.p", "wb" ) )

In [52]:
multiModelResults = ['MultiModel', round(testAccOva/nSplits,3), tnOva/nSplits, fpOva/nSplits, round(((tnOva/nSplits)/((tnOva/nSplits)+ (fpOva/nSplits))),3), fnOva/nSplits, tpOva/nSplits,round(((tpOva/nSplits)/((tpOva/nSplits)+ (fnOva/nSplits))),3),]

In [53]:
print(multiModelResults)

['MultiModel', 0.834, 961.2, 2.0, 0.998, 198.4, 42.4, 0.176]


In [54]:
ovaResults.append(multiModelResults)

# Results

In [55]:
allResults = pd.DataFrame(data = ovaResults, columns = ["Type","Acc", "TN", "FP", "TNR", "FN", "TP", "TPR"])

In [56]:
allResults

,Type,Acc,TN,FP,TNR,FN,TP,TPR
0,Majority,0.800,963.2,0.0,1.000,240.8,0.0,0.000
1,Rule,0.800,963.2,0.0,1.000,240.8,0.0,0.000
2,Text,0.808,962.8,0.4,1.000,230.2,10.6,0.044
3,MultiFeature,0.832,955.2,8.0,0.992,194.8,46.0,0.191
4,MultiModel,0.834,961.2,2.0,0.998,198.4,42.4,0.176


In [57]:
pickle.dump( allResults, open( "Pickles/BaselineSQS.p", "wb" ) )